# Interfaz de Usuario para Generador de Brochures
## Combinando el generador de brochures con una interfaz de Gradio

Este notebook combina la funcionalidad del generador de brochures del notebook 02 con una interfaz de usuario amigable usando Gradio del notebook 03.

### Importamos las librerías necesarias

In [ ]:
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from google import genai
from pydantic import BaseModel, Field, HttpUrl
from typing import List
import gradio as gr

### Cargamos las variables del fichero .env e inicializamos el cliente de Google Gemini

In [ ]:
load_dotenv()
client = genai.Client()

### Clase Website para scrappear sitios web

In [ ]:
class Website:
    """
    Una clase de utilidad para representar un sitio web que hemos scrappeado, ahora con enlaces
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "Sin título"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Título de la Web:\n{self.title}\nContenido de la Web:\n{self.text}\n\n"

### Configuración de prompts para obtener enlaces relevantes

In [ ]:
link_system_prompt = """Se te proporciona una lista de enlaces que se encuentran en una página web.
Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa,
como enlaces a una página Acerca de, una página de la empresa, las carreras/empleos disponibles o páginas de servicios.
Debes responder en JSON como en este ejemplo:
{
    "links": [
        {"type": "Pagina Sobre nosotros", "url": "https://url.completa/aqui/va/sobre/nosotros"},
        {"type": "Pagina de Seguros", "url": "https://otra.url.completa/seguros"}
    ]
}
"""

In [ ]:
def get_links_user_prompt(website):
    links_list = '\n'.join(website.links)
    user_prompt = f"""Aquí hay una lista de enlaces de la página web {website.url} - 
    Por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON.
    No incluyas Términos y Condiciones, Privacidad ni enlaces de correo electrónico.
    Links (puede que algunos sean links relativos): {links_list}"""
    return user_prompt

In [ ]:
class Link(BaseModel):
    type: str = Field(description="categoria del enlace de la página web, segun reelevancia para el folleto.")
    url: HttpUrl = Field(description="URL completa del enlace.")
    
class LinksList(BaseModel):
    links: List[Link] = Field(description="Lista de enlaces.")

def get_links_google(url):
    website = Website(url)
    response = client.models.generate_content(
        model="gemini-3-flash-preview",
        contents=[link_system_prompt, get_links_user_prompt(website)],
        config={
            "response_mime_type": "application/json",
            "response_json_schema": LinksList.model_json_schema(),
        },
    )

    return LinksList.model_validate_json(response.text).model_dump()

### Funciones para generar el brochure

In [ ]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    try:
        links = get_links_google(url)
        print("Links encontrados:", links)
        for link in links["links"]:
            result += f"\n\n{link['type']}\n"
            try:
                result += Website(link["url"]).get_contents()
            except:
                print(f"Error al acceder al enlace: {link['url']}")
    except Exception as e:
        print(f"Error al obtener enlaces: {e}")
    return result

In [ ]:
brochure_system_prompt = """Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa
y crea un folleto breve sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.
Incluye detalles sobre la cultura de la empresa, los clientes, las carreras/empleos y los cursos/packs para futuros empleos si tienes la información."""

In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""Estás mirando una empresa llamada: {company_name}. Aquí se encuentra el contenido de su página de inicio y otras páginas relevantes. 
                      Usa esta información para crear un breve folleto de la empresa en Markdown: {get_all_details(url)}"""
    return user_prompt

### Función principal para crear brochure con streaming

In [ ]:
def create_brochure_streaming(company_name, url, progress=gr.Progress()):
    """
    Crea un brochure con respuesta en streaming para una mejor experiencia de usuario
    """
    try:
        progress(0.1, desc="Analizando la página web...")
        
        user_prompt = get_brochure_user_prompt(company_name, url)
        
        progress(0.5, desc="Generando brochure...")
        
        response = client.models.generate_content_stream(
            model='gemini-3-flash-preview',
            contents=[brochure_system_prompt,user_prompt]
        )
        
        full_text = ""
        
        for chunk in response:
            full_text += chunk.text
            yield full_text
        
        progress(1.0, desc="¡Brochure completado!")
        
    except Exception as e:
        error_message = f"Error al generar el brochure: {str(e)}"
        yield error_message

### Interfaz de Gradio

In [ ]:
def create_brochure_interface():
    """
    Crea la interfaz de usuario de Gradio para el generador de brochures
    """
    with gr.Blocks(title="Generador de Brochures Empresariales", ) as demo:
        gr.Markdown(
            """
            # 📄 Generador de Brochures Empresariales
            
            Crea brochures profesionales para empresas analizando automáticamente su sitio web.
            
            **Instrucciones:**
            1. Ingresa el nombre de la empresa
            2. Proporciona la URL del sitio web de la empresa
            3. Haz clic en "Generar Brochure" y espera el resultado
            
            El sistema analizará la página principal y páginas relevantes para crear un brochure completo.
            """
        )
        
        with gr.Row():
            with gr.Column(scale=1):
                company_name = gr.Textbox(
                    label="Nombre de la Empresa",
                    placeholder="Ej: Google, Microsoft, OpenAI...",
                    value=""
                )
                
                company_url = gr.Textbox(
                    label="URL del Sitio Web",
                    placeholder="https://ejemplo.com",
                    value=""
                )
                
                generate_btn = gr.Button(
                    "🚀 Generar Brochure",
                    variant="primary",
                    size="lg"
                )
                
                gr.Examples(
                    examples=[
                        ["Aegon España", "https://www.aegon.es"],
                        ["Anthropic", "https://anthropic.com"],
                        ["Hugging Face", "https://huggingface.co"]

                    ],
                    inputs=[company_name, company_url],
                    label="Ejemplos para probar"
                )
            
            with gr.Column(scale=2):
                brochure_output = gr.Markdown(
                    label="Brochure Generado",
                    value="El brochure aparecerá aquí una vez que se genere...",
                    height=600
                )
        
        generate_btn.click(
            fn=create_brochure_streaming,
            inputs=[company_name, company_url],
            outputs=[brochure_output],
            show_progress=True
        )
    
    return demo

### Lanzar la interfaz

In [ ]:
demo = create_brochure_interface()
demo.launch(theme=gr.themes.Soft(),inline=False, share=False)  # Cambia a True si quieres compartir la interfaz públicamente

### Versión alternativa con interfaz más simple

In [ ]:
def simple_brochure_generator(company_name, url):
    """
    Simplified version for quick testing
    """
    try:
        user_prompt = get_brochure_user_prompt(company_name, url)
        
        response = client.models.generate_content(
            model='gemini-3-flash-preview',
            contents=[brochure_system_prompt,get_brochure_user_prompt(company_name, url)]
        )
        return response.text
        
    except Exception as e:
        return f"Error al generar el brochure: {str(e)}"

### Interfaz simple alternativa

In [ ]:
simple_interface = gr.Interface(
    fn=simple_brochure_generator,
    inputs=[
        gr.Textbox(label="Nombre de la Empresa", placeholder="Ej: Google"),
        gr.Textbox(label="URL del Sitio Web", placeholder="https://ejemplo.com")
    ],
    outputs=gr.Markdown(label="Brochure Generado"),
    title="🏢 Generador Simple de Brochures",
    description="Ingresa el nombre de la empresa y su URL para generar un brochure profesional.",
    examples=[
        ["Aegon España", "https://www.aegon.es"],
        ["Anthropic", "https://anthropic.com"],
        ["Hugging Face", "https://huggingface.co"]
    ],
    flagging_mode="never"
)
simple_interface.launch(inline=False, share=False)